
# 🎧 OpenAI TTS（自動分段 → 合併｜背景音可選｜.txt 上傳版）

**功能重點**
- 自動把長篇 `.txt` 依字數**切段**（預設每段 ≤ 1800 字，目標 1600）
- 每段用 `gpt-4o-mini-tts` 生成 MP3
- **自動合併**成一個完整 `narration_full.mp3`
- 可選**背景音混音**：有上傳就加（循環/裁切、淡入淡出、音量），沒上傳則略過
- API Key 以 `getpass()` 寫入 `os.environ["OPENAI_API_KEY"]`（不回顯）

> 提醒：TTS 模型單次輸入上限約 2000 tokens（粗估 1600–2000 字）。
> ⚠️ 注意：
> - 請自行在執行環境中設定 `OPENAI_API_KEY` 環境變數，請勿將實際金鑰寫入此 Notebook。
> - 本 Notebook 僅提供 TTS 與字幕處理流程示例，不包含完整故事文本。
> - 如需商用，請自行確認語音模型、授權與版權條款。


In [ ]:

# === 1) 安裝相依套件 ===
!pip -q install --upgrade openai pydub
!apt-get -y -qq install ffmpeg > /dev/null
print("✅ 已安裝 openai, pydub, ffmpeg")


✅ 已安裝 openai, pydub, ffmpeg



## 🔐 2) 設定 API Key（不顯示、僅本次會話）


In [ ]:

import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("請輸入你的 OpenAI API Key：")
assert os.environ.get("OPENAI_API_KEY"), "❌ 尚未設定 OPENAI_API_KEY"
print("🔑 已設定 OPENAI_API_KEY 到環境變數")


請輸入你的 OpenAI API Key：··········
🔑 已設定 OPENAI_API_KEY 到環境變數



## 📤 3) 上傳你的故事 `.txt`
- 建議 UTF-8 編碼；會顯示前 500 字預覽 & 字數


In [ ]:

from google.colab import files

print("請選擇你的故事 .txt 檔…")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("未選擇任何檔案。")

txt_name = next(iter(uploaded))
STORY_RAW = uploaded[txt_name].decode('utf-8', errors='replace')

print(f"✅ 已讀取：{txt_name}")
print("📏 字元數 =", len(STORY_RAW))
print("\n—— 前 500 字預覽 ——\n")
print(STORY_RAW[:500])


請選擇你的故事 .txt 檔…


Saving 東方式靈異故事集_純朗讀稿.txt to 東方式靈異故事集_純朗讀稿.txt
✅ 已讀取：東方式靈異故事集_純朗讀稿.txt
📏 字元數 = 7663

—— 前 500 字預覽 ——

東方式靈異故事集（純朗讀稿）

故事一：山道迴音

我其實不太想講這個，因為講了之後，通常隔天晚上它就會來確認一下我還記不記得它。我不是威脅你啦，先別緊張，我都還活得好好的，只是那個聲音現在偶爾還會在我家樓梯間出現。

好，我講。

事情是去年端午連假，我跟朋友去宜蘭山上露營。那邊是半開放營地，老闆就一對中年夫妻，房子在下方，我們搭帳篷在比較高的一層平台。平台邊緣有護繩，可是再往外就直接是陡下去的山谷，空氣很濕，松針踩下去會汩一聲，好像踩到有彈性的地毯。

我們四個人，兩男兩女，感情都不錯。說真的我原本根本不想去露營，我不喜歡洗不到熱水澡，不喜歡昆蟲，不喜歡濕睡袋黏大腿那種觸感，真的很煩。但是他們就說「上班族真的會廢掉欸」、「出去走一下嘛」、「你整天坐冷氣房會生病啦」，我就跟。

第一天白天都很正常，煮東西，烤肉，喝啤酒，大家拍照，上傳限動。到晚上十點多的時候，山霧開始上來，空氣變得很安靜。那種安靜是連蟲鳴都被棉被蓋住的感覺，靜到你會開始聽到自己吞口水。

霧上來之後，阿哲就跑去上廁所。廁所在比較下面，靠近老闆家的地方，要走階梯路下去，大概一百公尺。不遠，可是下去的時候燈不多，只有幾盞



## 🧹 4) （可選）清理舞台標記 → 產生純旁白文本
- 會移除括號中的提示字（如：（短停）、(輕嘆)、【…】、[...] 等）
- 若你的文本已是純旁白，可直接執行，不會影響內容（只會清掉標記）


In [ ]:

import re

def clean_to_narration(text: str) -> str:
    t = text
    t = re.sub(r"（[^）]*）", "", t)   # 全形括號
    t = re.sub(r"\([^)]*\)", "", t) # 半形括號
    t = re.sub(r"【[^】]*】", "", t)   # 全形中括
    t = re.sub(r"\[[^\]]*\]", "", t) # 半形中括
    # 移除純分隔線或 '——以下' 行
    lines = []
    for line in t.splitlines():
        s = line.strip()
        if not s:
            lines.append("")
            continue
        if s.startswith("——") or "——以下" in s:
            continue
        if set(s) <= set("-—–"):
            continue
        lines.append(line)
    t = "\n".join(lines)
    # 合併多餘空白行
    t = re.sub(r"\n{3,}", "\n\n", t).strip()
    return t

STORY = clean_to_narration(STORY_RAW)
print("🧽 清理完成。清理前後長度：", len(STORY_RAW), "→", len(STORY))


🧽 清理完成。清理前後長度： 7663 → 7654



## ✂️ 5) 自動分段（避免超過 TTS 單次上限）
- 預設 **目標 1600 字**、**最大 1800 字**
- 優先在 **句號/頓號/換行** 等斷點切分


In [ ]:

import re

TARGET_LEN = 1600
MAX_LEN = 1800

# 以標點切句（包含中英文標點與換行）
SENT_SPLIT = re.compile(r"([^\n。！？!?；;：:、，,]+[\n。！？!?；;：:、，,]*)")

def split_sentences(text: str):
    # 保留標點在句尾
    parts = SENT_SPLIT.findall(text)
    if not parts:
        return [text]
    # 合併零碎（例如空白）
    merged = []
    buf = ""
    for p in parts:
        buf += p
        if p.endswith(("\n","。","！","？","!","?","；",";","：",":","、","，",",")):
            merged.append(buf)
            buf = ""
    if buf:
        merged.append(buf)
    return [s for s in merged if s.strip()]

def chunkify(text: str, target=TARGET_LEN, maxlen=MAX_LEN):
    sents = split_sentences(text)
    chunks, buf = [], ""
    for s in sents:
        if len(buf) + len(s) <= maxlen:
            buf += s
        else:
            if buf:
                chunks.append(buf)
                buf = ""
            # 單句過長則硬切
            while len(s) > maxlen:
                chunks.append(s[:maxlen])
                s = s[maxlen:]
            buf = s
    if buf:
        chunks.append(buf)
    # 第二遍微調：把太短的尾段盡量與前段合併至 target 附近
    tuned = []
    for seg in chunks:
        if not tuned:
            tuned.append(seg)
            continue
        if len(tuned[-1]) < target and len(tuned[-1]) + len(seg) <= maxlen:
            tuned[-1] += seg
        else:
            tuned.append(seg)
    return tuned

chunks = chunkify(STORY)
print(f"✅ 已分段：{len(chunks)} 段")
for i,c in enumerate(chunks[:5]):
    print(f"第 {i+1} 段長度：{len(c)}")
if len(chunks) > 5:
    print("…（僅示前 5 段）")


✅ 已分段：5 段
第 1 段長度：1798
第 2 段長度：1786
第 3 段長度：1795
第 4 段長度：1799
第 5 段長度：476



## 💰 6) 費用試算（可調整單價）
- 預設：`0.015 USD / 1000 chars`


In [ ]:

RATE_USD_PER_KCHARS = 0.015
total_chars = sum(len(c) for c in chunks)
cost_usd = (total_chars / 1000.0) * RATE_USD_PER_KCHARS
print(f"📊 總字數：{total_chars}")
print(f"💵 預估成本（USD）：{cost_usd:.4f}")


📊 總字數：7654
💵 預估成本（USD）：0.1148



## 🔊 7) 逐段 TTS 生成 MP3（自動重試）
- 模型：`gpt-4o-mini-tts`
- 聲線：`shimmer`


In [ ]:

import os, time, math
from openai import OpenAI
from IPython.display import Audio, display

MODEL = "gpt-4o-mini-tts"
VOICE = "shimmer"

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

part_paths = []
for idx, seg in enumerate(chunks, start=1):
    outp = f"part_{idx:02}.mp3"
    # 簡單重試機制
    for attempt in range(4):
        try:
            with client.audio.speech.with_streaming_response.create(
                model=MODEL,
                voice=VOICE,
                input=seg
            ) as response:
                response.stream_to_file(outp)
            print(f"✅ 產生完成：{outp}（第 {idx}/{len(chunks)} 段, {len(seg)} 字）")
            part_paths.append(outp)
            break
        except Exception as e:
            wait = 2 ** attempt
            print(f"⚠️ 失敗重試（{attempt+1}/4），{wait}s 後重試。原因：{e}")
            time.sleep(wait)
    else:
        raise RuntimeError(f"❌ 無法產生第 {idx} 段 MP3，請稍後再試。")

# 播放第一段預覽
if part_paths:
    display(Audio(part_paths[0]))



## 🔗 8) 合併所有段落 → `narration_full.mp3`


In [ ]:

from pydub import AudioSegment

assert part_paths, "沒有可合併的段落"
full = AudioSegment.silent(duration=0)
for p in part_paths:
    full += AudioSegment.from_file(p)

NARRATION_FULL = "narration_full.mp3"
full.export(NARRATION_FULL, format="mp3", bitrate="128k")
print("🎉 已輸出完整旁白 ->", NARRATION_FULL)


🎉 已輸出完整旁白 -> narration_full.mp3



## 🎼 9) （可選）上傳背景音並混音
- 沒上傳：跳過混音，直接使用 `narration_full.mp3`
- 有上傳：自動循環/裁切 + 淡入/淡出 + 音量設定，輸出 `final_story.mp3`


In [ ]:

from google.colab import files
from pydub import AudioSegment
from IPython.display import Audio, display

BG_GAIN_DB = -18   # 背景相對音量（-12 比較明顯、-20 更輕）
FADE_IN_MS = 3000
FADE_OUT_MS = 4000

print("（可選）上傳背景音 .mp3/.wav；若不需要可跳過此格")
try:
    bg_uploaded = files.upload()
except Exception:
    bg_uploaded = {}

if not bg_uploaded:
    FINAL_OUT = "narration_full.mp3"
    print("ℹ️ 未上傳背景音 → 直接使用旁白：", FINAL_OUT)
else:
    bg_name = next(iter(bg_uploaded))
    narr = AudioSegment.from_file("narration_full.mp3")
    bg = AudioSegment.from_file(bg_name)
    if len(bg) == 0:
        FINAL_OUT = "narration_full.mp3"
        print("⚠️ 背景音長度為 0 → 略過混音，使用旁白：", FINAL_OUT)
    else:
        # 循環/裁切到與旁白一致長度
        looped = AudioSegment.silent(duration=0, frame_rate=bg.frame_rate)
        while len(looped) < len(narr):
            looped += bg
        bg = looped[:len(narr)]
        # 音量與淡化
        bg = bg + BG_GAIN_DB
        bg = bg.fade_in(FADE_IN_MS).fade_out(FADE_OUT_MS)
        # 疊合
        mixed = bg.overlay(narr)
        FINAL_OUT = "final_story.mp3"
        mixed.export(FINAL_OUT, format="mp3", bitrate="128k")
        print("🎧 混音完成 ->", FINAL_OUT)

# 播放最終輸出
display(Audio(FINAL_OUT))



## 💾 10) 下載（最終輸出檔）
- 若未上傳背景音：下載 `narration_full.mp3`
- 若已混音：下載 `final_story.mp3`


In [ ]:

from google.colab import files
files.download("final_story.mp3" if __import__("os").path.exists("final_story.mp3") else "narration_full.mp3")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>